# Chap.04 スクレイピングの基本

## 01 ライブラリのインストール

## 02 Webページをスクレイピングする

### ライブラリをインストールする

- Requests
- lxml
- cssselect

### Web上のリソースを取得する

- RequestsでWeb上のリソースを取得する
- REquestsのレスポンス
- JSONのパース

### XPathとCSSセレクター

- XPathで要素を指定する
- CSSセレクターで要素を指定する

### HTMLソースを解析する

- 抜き出したい要素のXPathとCSSセレクターを取得する
- lxmlでスクレイピング

## 03 RSSをスクレイピングする

### ライブラリのインストール

- feedparser
- XMLを解析する

## 04 データをデータベースに保存して解析する

### データをデータベースに保存して呼び出す

### MySQLのインストール

- `brew install mysql`
- `mysql --version`
- `mysql.server start`

### データベースとユーザーを作成する

- rootユーザーとして、ユーザーを指定し、パスワードを指定しない。
  - `mysql -u root -p`
  - パスワードを入力せず、`[Enter]`
- データベースとの接続を解除する
  - `exit`
  - `[Ctrl] + [D]`
- データベースの作成
  - データベース名: scrapingdata
  - 文字コード: utf8
  - `CREATE DATABASE scrapingdata DEFAULT CHARACTER SET utf8;`
- ユーザーの作成
  - ユーザー名: scrapingman
  - パスワード: myPassword-1
  - `CREATE USER "scrapingman" IDENTIFIED BY "myPassword-1"`
- ユーザーscrapingmanに読み書きの権限を与える
  - `GRANT ALL ON scrapingdata.* TO scrapingman;`

### PythonからMySQLに接続する

- mysqlclientのインストール
  - `pip install mysqlclient`

&nbsp;

```python
import MySQLdb

# データベースに接続する
connection = MySQLdb.connect(
    user="scrapingman",
    passwd="myPassword-1",
    host="localhost",
    db="scrapingdata",
    charset="utf-8"
)

# カーソルを生成する
cursor = connection.cursor()

# 実行するたびに同じ結果なるようにテーブルを削除しておく
cursor.execute("DROP TABLE IF EXISTS books")

# テーブルを作成する
cursor.execute("CREATE TABLE books (title text, url text)")

# データを保存する
cursor.execute(
    "INSERT INTO books VALUES(%s, %s)",
    ("はじめてのPython", "https://example.com")
)

# 変更をコミットする
connection.commit()

# 接続を閉じる
connection.close()
```

&nbsp;

```python
# connect関数の戻り値を確認する
type(connection)

# カーソルを確認する
type(cursor)
```

&nbsp;

- テーブルを確認する

```bash
mysql -u root -p
```

```bash
mysql> use scrapingdata;
```

```bash
mysql> select * from books;
```

```bash
mysql> exit
```

### 解析した結果を保存する

```python
# chap04_04.py
import feedparser
import MySQLdb

# データベースに接続する
connection = MySQLdb.connect(
    user="scrapingman",
    passwd="myPassword-1",
    host="localhost",
    db="scrapingdata",
    charset="utf-8"
)

# カーソルを生成する
cursor = connection.cursor()

# 実行するたびに同じ結果になるようにテーブルを削除しておく
cursor.execute("DROP TABLE IF EXISTS books")

# テーブルを作成する
cursor.execute("CREATE TABLE books (title text, url text)")

# URLを指定してFeedParserDictオブジェクトを取得する
rss = feedparser.parse("https://www.shoeisha.co.jp/rss/index.html")

# RSSのバージョンを表示する
print(rss.version)

# フィードタイトルとコンテンツの発行日時を表示する
print(rss["feed"]["title"])
print(rss["feed"]["published"])

# 各エントリーのタイトルとリンクを表示する
for content in rss["entries"]:
    # データを保存する
    cursor.execute(
        "INSERT INTO books VALUES(%s, %s)",
        (content["title"], content["link"])
    )

# 変更をコミットする
connection.commit()

# 接続を閉じる
connection.close()
```

### シェルからスクリプトから実行

```bash
# スクリプトの実行
python chap04_04.py
```

```bash
# MySQLの接続
mysql -u root -p
```

```bash
mysql> use scrapingdata;
```

```bash
mysql> select * from books:
```